In [2]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [27]:
import os
os.chdir("D:\各种材料\研究生\公众号\三、美林时钟模型")
data=pd.read_excel(r"D:\各种材料\研究生\公众号\三、美林时钟模型\模型改进的探索.xlsx")
data=data.dropna(0)
data.index=data.日期
del data["日期"]
data.columns=["M2","CPI","PPI","Eco_Same","Eco_Ahead","Dollar_Index","Interest_rate","CPI_PPI"]
data.head()

,M2,CPI,PPI,Eco_Same,Eco_Ahead,Dollar_Index,Interest_rate,CPI_PPI
日期,,,,,,,,
2002-01-31,13.1,-1.0,-4.20,94.93,100.72,-5.00,2.2285,3.20
2002-02-28,13.0,0.0,-4.20,95.01,100.75,-4.70,1.9435,4.20
2002-03-31,14.4,-0.8,-4.00,95.45,100.67,-4.50,1.7682,3.20
2002-04-30,14.1,-1.3,-3.06,96.54,100.71,-3.38,1.8773,1.76
2002-05-31,14.0,-1.1,-2.63,96.98,100.82,-2.74,1.7988,1.53


In [29]:
df=(data-data.mean())/data.std()#数据标准化
est = smf.ols(formula='CPI ~  M2', data=df).fit() #前面的字符是显示了自变量与因变量
res=est.summary2()
res

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.009   
Dependent Variable: CPI              AIC:                607.3982
Date:               2020-02-08 23:27 BIC:                614.1301
No. Observations:   214              Log-Likelihood:     -301.70 
Df Model:           1                F-statistic:        2.898   
Df Residuals:       212              Prob (F-statistic): 0.0902  
R-squared:          0.013            Scale:              0.99117 
------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025  0.975]
------------------------------------------------------------------
Intercept       0.0000    0.0681   0.0000  1.0000  -0.1342  0.1342
M2             -0.1161    0.0682  -1.7022  0.0902  -0.2506  0.0183
-----------------------------------------------------------------
Omnibus:              25.456       Durbin-Watson:          0.090 
Prob(Omnibus):        0.000        Jarque-Bera (JB):       31.298
Skew:                 0.829        Prob(JB):               0.000 
Kurtosis:             3.874        Condition No.:          1     
=================================================================

"""

<font color="08#0000" size=4 face="宋体">**滞后阶数检验**</font>


In [34]:
def Lag_Term(D,Vf,Vb,T):
    #D is the DataFrame, T is the lag term.
    #Vb(因变量),Vf(自变量)分别对应于滞后的，和提前的两隔经济变量的名字
    data_b=D[Vb]
    reg_b=data_b[T:].values
    data_f=D[Vf]
    if (T >0):
        reg_f=data_f[:-T].values
    else:
        reg_f=data_f[:].values 
    L=len(reg_b)
    reg_temp=np.array([reg_b,reg_f])
    reg_temp=reg_temp.T
    reg_final=pd.DataFrame(reg_temp,columns=[Vb,Vf])
    R=reg_final[Vb].corr(reg_final[Vf])
    est = smf.ols(formula=Vb+"~"+Vf , data=reg_final).fit() #前面的字符是显示了自变量与因变量
    res=est.summary2()
    return res,R

In [42]:
Lag_Term(data,"PPI","CPI",8)

(<class 'statsmodels.iolib.summary2.Summary'>
 """
                  Results: Ordinary least squares
 Model:              OLS              Adj. R-squared:     0.037   
 Dependent Variable: CPI              AIC:                849.1107
 Date:               2020-02-08 23:38 BIC:                855.7664
 No. Observations:   206              Log-Likelihood:     -422.56 
 Df Model:           1                F-statistic:        8.795   
 Df Residuals:       204              Prob (F-statistic): 0.00338 
 R-squared:          0.041            Scale:              3.5764  
 -------------------------------------------------------------------
              Coef.    Std.Err.      t      P>|t|    [0.025   0.975]
 -------------------------------------------------------------------
 Intercept    2.3803     0.1408   16.9048   0.0000   2.1027   2.6579
 PPI          0.0899     0.0303    2.9657   0.0034   0.0301   0.1496
 -----------------------------------------------------------------
 Omnibus:         